In [ ]:
import httpx

In [ ]:
def snomed_browser(term: str, limit: int = 100, semanticTag: str = "procedure") -> list[dict]:
    """
    Search SNOMED CT for concepts matching the given term.
    """
    base_url = "http://snowstorm:8080"
    edition = "MAIN"

    url = f"{base_url}/{edition}/concepts"

    params = {
        # "term": term,
        "activeFilter": "true",  # recommended filter by SNOMED CT
        "termActive": "true",  # recommended filter by SNOMED CT
        "ecl": f'< 71388002|Procedure| {{{{ term = ("{term}") }}}}',
    }

    print(params)
    with httpx.Client() as client:
        response = client.get(url, params=params)
        response.raise_for_status()
        items = response.json().get("items", [])
        filtered_matches = [
            match
            for match in items
            # if match["definitionStatus"] == "FULLY_DEFINED"
        ]
        filtered_matches = sorted(
            filtered_matches,
            key=lambda x: len(x["fsn"]["term"]),
        )
        return filtered_matches[:limit]

In [ ]:
search_term = "Electroencephalogram"
matches = snomed_browser(search_term)

matches

In [ ]:
from medminer.tools.procedure import SNOMEDTool

tool = SNOMEDTool(base_url="http://snowstorm:8080", edition="MAIN")

In [ ]:
for q in tool._build_ecl_queries("Computed Cranial Tomography", {'Cranial': 'Head'}, ["cCT"]):
    print(q)

In [ ]:
tool.forward("Computed Cranial Tomography", {'Cranial': 'Head'}, ["cCT"])